In [19]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue

import numpy as np

In [2]:
client = QdrantClient(host="localhost", port=6333)

In [ ]:
#CREAR COLLECCION
client.create_collection(collection_name="coleccion_climas",
    vectors_config=VectorParams(size=7, distance=Distance.MANHATTAN,
                                datatype='uint8'))

True

In [9]:
#INSERTAR VECTORES
vectors = np.array([[32, 34, 33, 36, 33, 32, 35],
                    [33, 34, 36, 37, 34, 32, 34],
                    [21, 23, 24, 22, 22, 21, 19],
                    [10, 8, 12, 12, 9, 10, 10],
                    [19, 21, 20, 18, 21, 22, 20],
                    [31, 29, 30, 32, 34, 33, 31],
                    [24, 22, 21, 21, 21, 23, 22],
                    [7, 6, 10, 11, 8, 9, 7]])

array_idx = [1, 2, 3, 4, 5, 6, 7, 8]
array_payload = [{'region': 'norte', 'ciudad': 'monterrey'},
                 {'region': 'sur', 'ciudad': 'merida'},
                 {'region': 'centro', 'ciudad': 'cdmx'},
                 {'region': 'centro', 'ciudad': 'toluca'},
                 {'region': 'centro', 'ciudad': 'pachuca'},
                 {'region': 'norte', 'ciudad': 'chihuahua'},
                 {'region': 'sur', 'ciudad': 'tuxtla'},
                 {'region': 'centro', 'ciudad': 'metepec'}]

print(vectors.shape)

(8, 7)


In [10]:
for i in range(len(array_idx)):
    client.upsert(
       collection_name="coleccion_climas",
       points=[
          PointStruct(
                id=array_idx[i],
                vector=vectors[i].tolist(),
                payload=array_payload[i]
          )
       ]
    )

In [15]:
#SIMPLE QUERY
query_vector = np.array([16, 19, 20, 18, 18, 17, 21])
hits = client.search(
   collection_name="coleccion_climas",
   query_vector=query_vector,
   limit=3  # Return 5 closest points
)

print(hits)

[ScoredPoint(id=5, version=4, score=14.0, payload={'region': 'centro', 'ciudad': 'pachuca'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=7, version=6, score=25.0, payload={'region': 'sur', 'ciudad': 'tuxtla'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=2, score=27.0, payload={'region': 'centro', 'ciudad': 'cdmx'}, vector=None, shard_key=None, order_value=None)]


/tmp/ipykernel_110613/373973628.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


In [20]:
#FILTER QUERY
query_vector = np.array([16, 19, 20, 18, 18, 17, 21])
hits = client.search(
   collection_name="coleccion_climas",
   query_vector=query_vector,
   limit=3,
   query_filter=Filter(
      must=[  # These conditions are required for search results
            FieldCondition(key='region', match=MatchValue(value='sur'))
      ]
   ),
)

print(hits)

[ScoredPoint(id=7, version=6, score=25.0, payload={'region': 'sur', 'ciudad': 'tuxtla'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=1, score=111.0, payload={'region': 'sur', 'ciudad': 'merida'}, vector=None, shard_key=None, order_value=None)]


/tmp/ipykernel_110613/884208451.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


In [27]:
#QUERY
query_vector = np.array([16, 19, 20, 18, 18, 17, 21])
hits = client.query_points(
   collection_name="coleccion_climas",
   query=query_vector,
   limit=2
)

print(hits)

points=[ScoredPoint(id=5, version=4, score=14.0, payload={'region': 'centro', 'ciudad': 'pachuca'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=7, version=6, score=25.0, payload={'region': 'sur', 'ciudad': 'tuxtla'}, vector=None, shard_key=None, order_value=None)]
